In [192]:
%serialconnect



***Connecton broken [Input/output error]
You may need to reconnectserial exception on close write failed: [Errno 5] Input/output error

Closing serial Serial<id=0x7f47c1b60cc0, open=True>(port='/dev/ttyUSB0', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
Connecting to Serial /dev/ttyUSB1 baud=115200 
Ready.

In [195]:
# run this cell only once!  (Press reset button if you accidentally do it twice)
# There is a bug in ESP32-Micropython https://github.com/micropython/micropython-esp32/issues/208)

import machine
m19 = machine.PWM(machine.Pin(19), freq=50, duty=int(1.5/20*1024))
m18 = machine.PWM(machine.Pin(18), freq=50, duty=int(1.5/20*1024))
m5 = machine.PWM(machine.Pin(5), freq=50, duty=int(1.5/20*1024))
m21 = machine.PWM(machine.Pin(21), freq=50, duty=int(1.5/20*1024))
print(m19, m18, m5, m21)


PWM(19, freq=50, duty=16992) PWM(18, freq=50, duty=16992) PWM(5, freq=50, duty=16992) PWM(21, freq=50, duty=16992)


In [196]:
print(m19, m18, m5, m21)

PWM(19, freq=50, duty=16992) PWM(18, freq=50, duty=16992) PWM(5, freq=50, duty=16992) PWM(21, freq=50, duty=16992)


In [180]:
import machine

# Set up the four possible input pulse pins 
a34 = machine.ADC(machine.Pin(34))
a34.atten(machine.ADC.ATTN_11DB)
a35 = machine.ADC(machine.Pin(35))
a35.atten(machine.ADC.ATTN_11DB)

a32 = machine.ADC(machine.Pin(32))
a32.atten(machine.ADC.ATTN_11DB)
a33 = machine.ADC(machine.Pin(33))
a33.atten(machine.ADC.ATTN_11DB)


In [181]:
# lists of pins to read, and servo pins to output for immediate noisy feedback
# (you can shorten or lengthen these lists if you are not using all the pulse sensors)
ads = [a34,a32,a35,a33]
sds = [m19,m18,m5,m21]


In [182]:
# Full function for reading the sets of pulse sensors
import time

def measure34sub(numberofseconds, samplespersecond, subsamplestoaverage, compensationratio):
    ticks0 = time.ticks_ms()
    
    # compensationratio is used to shorten the sleep cycle so that we hit the right time durations
    # (by sleeping slightly less than we should because the rest of the code is slow)
    microsecondstep = int(1/samplespersecond/subsamplestoaverage*1000000*compensationratio)
    
    # one second per line
    for i in range(numberofseconds):
        
        # readings per second
        for j in range(samplespersecond):
            
            # number of samples averaged per (1/samplespersecond)^th of a second for a reading
            samplesums = [0]*len(ads)
            for k in range(subsamplestoaverage):
                
                # make a reading from each of the ADC
                for i, a in enumerate(ads):
                    samplesums[i] += a.read()
                time.sleep_us(microsecondstep)
            print(",".join(str(int(samplesum/subsamplestoaverage+0.5))  for samplesum in samplesums), end=" ")

            for i in range(min(len(ads), len(sds))):
                v = int(samplesums[i]/subsamplestoaverage+0.5)
                v = (v-1700)/(1900-1700)
                v = max(0.1, min(0.9, v))
                sds[i].duty(int((v*2 + 0.5)/20*1024))
                
        print()
    return (time.ticks_ms() - ticks0)/1000
    

In [ ]:
# the cell below runs and captures 100 lines (100 seconds) of data into the file "testpulsemulti50avg.txt"

# ** Use Menu: Kernel->Interrupt to break early

In [190]:
%capture --quiet testpulsemulti50avg.txt
elapsedtime = measure34sub(numberofseconds=100, samplespersecond=50, 
                           subsamplestoaverage=10, compensationratio=0.7)

# compensationratio shortens the sleep periods to try and make the 
# elapsedtime equal to numberofseconds.  
# (It compensates for python running slow by sampling slightly more often)

85 lines captured

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
  File "<stdin>", line 24, in measure34sub
  File "<stdin>", line 24, in <genexpr>
KeyboardInterrupt: 


In [188]:
print(elapsedtime)

120.053
